In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


#  **Loading the Data**

In [2]:
data_train=pd.read_csv('/kaggle/input/titanic/train.csv')
data_test=pd.read_csv('/kaggle/input/titanic/test.csv')
print(data_train.info)

<bound method DataFrame.info of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                   

# **Determing Necessary Features**


In [3]:
men=data_train.loc[(data_train.Sex=='male')]
men_survived=data_train.loc[(data_train.Sex=='male') & (data_train.Survived==1)]

women=data_train.loc[(data_train.Sex=='female')]
women_survived=data_train.loc[(data_train.Sex=='female') & (data_train.Survived==1)]

print(len(men_survived)/len(men)*100)
print(len(women_survived)/len(women)*100)

18.890814558058924
74.20382165605095


<span style="font-size: 18px;">Shipwreck cases require women and children to be evacuated before men, therefore survival rate of women is more than men.
This can be an essential feature.
</span>

In [4]:
children=data_train.loc[(data_train.Age<18)]
children_survived=data_train.loc[(data_train.Age<18) & (data_train.Survived==1)]
print((len(children_survived)/len(children)))

adult=data_train.loc[(data_train.Age>18) & (data_train.Age<65)]
adult_survived=data_train.loc[(data_train.Age>18) & (data_train.Age<65) & (data_train.Survived==1)]
print((len(adult_survived)/len(adult)))

elder=data_train.loc[(data_train.Age>65)]
elder_survived=data_train.loc[(data_train.Age>65) & (data_train.Survived==1)]
print((len(elder_survived)/len(elder)))

0.5398230088495575
0.3882978723404255
0.125


<span style="font-size: 18px;">There is also a noticeable difference between survival of different age groups.</span>

In [5]:
girl_children=data_train.loc[(data_train.Age<18) & (data_train.Sex=='female')]
girl_children_survived=data_train.loc[(data_train.Age<18) & (data_train.Sex=='female') & (data_train.Survived==1)]
print((len(girl_children_survived)/len(girl_children)))

boy_children=data_train.loc[(data_train.Age<18) & (data_train.Sex=='male')]
boy_children_survived=data_train.loc[(data_train.Age<18) & (data_train.Sex=='male') & (data_train.Survived==1)]
print((len(boy_children_survived)/len(boy_children)))

men_adult=data_train.loc[(data_train.Age>18) & (data_train.Sex=='male') & (data_train.Age<65)]
men_adult_survived=data_train.loc[(data_train.Age>18) & (data_train.Sex=='male') & (data_train.Age<65) & (data_train.Survived==1)]
print((len(men_adult_survived)/len(men_adult)))

women_adult=data_train.loc[(data_train.Age>18) & (data_train.Sex=='female') & (data_train.Age<65)]
women_adult_survived=data_train.loc[(data_train.Age>18) & (data_train.Sex=='female') & (data_train.Age<65) & (data_train.Survived==1)]
print((len(women_adult_survived)/len(women_adult)))

men_elder=data_train.loc[(data_train.Age>65) & (data_train.Sex=='male')]
men_elder_survived=data_train.loc[(data_train.Age>65) & (data_train.Sex=='male') & (data_train.Survived==1)]
print((len(men_elder_survived)/len(men_elder)))

women_elder=data_train.loc[(data_train.Age>65) & (data_train.Sex=='female')]
print(len(women_elder))

0.6909090909090909
0.39655172413793105
0.18328840970350405
0.7823834196891192
0.125
0


<span style="font-size: 18px;">Interestingly, combining age and gender also produces some important observations.</span>

In [6]:
class1=data_train.loc[(data_train.Pclass==1)]
class1_survived=data_train.loc[(data_train.Pclass==1) & (data_train.Survived==1)]
print(len(class1_survived)/len(class1))

class2=data_train.loc[(data_train.Pclass==2)]
class2_survived=data_train.loc[(data_train.Pclass==2) & (data_train.Survived==1)]
print(len(class2_survived)/len(class2))

class3=data_train.loc[(data_train.Pclass==3)]
class3_survived=data_train.loc[(data_train.Pclass==3) & (data_train.Survived==1)]
print(len(class3_survived)/len(class3))

0.6296296296296297
0.47282608695652173
0.24236252545824846


<span style="font-size: 18px;">Class of a passenger is also important as class 1 passengers are usually evacuated first.</span>

In [7]:
sib_sp=data_train.loc[(data_train.SibSp==1)]
sib_sp_survived=data_train.loc[(data_train.SibSp==1) & (data_train.Survived==1)]
print(len(sib_sp_survived)/len(sib_sp))
print(1-(len(sib_sp_survived)/len(sib_sp)))

0.5358851674641149
0.46411483253588515


<span style="font-size: 18px;">Whether or not a passenger is travelling with a sibling or spouse can also be a candidate for a feature.</span>

# **Feature Engineering**
<span style="font-size: 18px;">Now, A class will be assigned to a passenger based on the ticket they purchase which can be engineered together as a socio-economic factor while evacuation.
<br><br>
Adding the sibling+parents+children can also be created into a new feature.
</span>

In [8]:
data_train['FarePclass'] = data_train['Fare'] + data_train['Pclass']
data_test['FarePclass'] = data_test['Fare'] + data_test['Pclass']

data_train['FamilySize'] = data_train['SibSp'] + data_train['Parch'] + 1
data_test['FamilySize'] = data_test['SibSp'] + data_test['Parch'] + 1

<span style="font-size: 18px;">Checking null values.</span>

In [9]:
print(data_train.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
FarePclass       0
FamilySize       0
dtype: int64


<span style="font-size: 18px;">filling null values based on median.</span>

In [10]:
median_train=data_train['Age'].median()
median_test=data_test['Age'].median()

data_train['Age'].fillna(median_train, inplace=True)
data_test['Age'].fillna(median_test, inplace=True)

medianfp_train=data_train['FarePclass'].median()
medianfp_test=data_test['FarePclass'].median()

data_train['FarePclass'].fillna(medianfp_train, inplace=True)
data_test['FarePclass'].fillna(medianfp_test, inplace=True)

In [11]:
features=['Sex','Age','Pclass','FamilySize','FarePclass']
x=pd.get_dummies(data_train[features])
y=data_train['Survived']
x_test_a=pd.get_dummies(data_test[features])
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
lr=LogisticRegression(max_iter=10000)
lr.fit(x_train,y_train)
predi = lr.predict(x_train)
acci = accuracy_score(predi,y_train)
print('accuracy =', acci*100)
y_pred=lr.predict(x_test)
ypred=lr.predict(x_test_a)
accsplit = accuracy_score(y_pred,y_test)
print('accuracy for split dataset =', accsplit*100)

accuracy = 80.19662921348315
accuracy for split dataset = 80.44692737430168


In [12]:
submission = pd.DataFrame({
    'PassengerId': data_test['PassengerId'],
    'Survived': ypred
})
submission.to_csv('/kaggle/working/submission.csv', index=False)